In [19]:
import requests
import re
import json
import MySQLdb

db = MySQLdb.connect(
    host="172.99.106.245",
    user="root",
    passwd="K0llaP4ss",
    db="poi2")

db.set_character_set('utf8')
dbc = db.cursor()
dbc.execute('SET NAMES utf8;')
dbc.execute('SET CHARACTER SET utf8;')
dbc.execute('SET character_set_connection=utf8;')
dbc.close()

TAG_RE = re.compile(r'<[^>]+>')


def remove_tags(text):
    return TAG_RE.sub('', text)

def req(p):

  r = requests.get('https://en.wikipedia.org/w/api.php', params=p)
  #print(TAG_RE.sub('', json.loads(r.text)['query']['pages'][p['pageids']]['extract']))
  #print(r.text)
  tagless = TAG_RE.sub('', json.loads(r.text)['query']['pages'][str(p['pageids'])]['extract'])
  #print(tagless)
  return {'pageid': p['pageids'], 'body': tagless}

def queryPageIds():
    cur = db.cursor()
    pageids = []
    or_w = (42.8470881, -124.7595306)
    or_s = (41.993219, -121.259210)
    or_e = (45.595339, -116.409837)
    or_n = (46.306281, -124.011349)
    or_coords = {
        'or_w': or_w[1],
        'or_e': or_e[1],
        'or_s': or_s[0],
        'or_n': or_n[0],
    }
    print or_coords
    query = "select gt_page_id from geo_tags where gt_lon < {or_e} and gt_lon > {or_w} and gt_lat > {or_s} and gt_lat < {or_n} and gt_region='OR'".format(
        **or_coords)

    cur.execute(query)
    or_poi = cur.fetchall()
    for x in or_poi:
        pageids.append(x[0])
    cur.close()
    return pageids


def createTableLocations():
    cur = db.cursor()
    query = "use poi2; create table IF NOT EXISTS locations (title varchar(70),body TEXT, pageid int PRIMARY KEY);"
    cur.execute(query)
    cur.close()

def updateLocation(p):
    cur = db.cursor()
    pageid = int(p['pageid'])
    query = "use poi2; INSERT INTO locations (pageid,body) VALUES (%s, %s) ON DUPLICATE KEY \
    UPDATE pageid = VALUES(pageid), body = VALUES(body)"
    result =  cur.execute(query, (pageid, p['body']))
    cur.close()
    db.commit()

    
## main ##

#queryPageIds()
createTableLocations()
pageids = list(set(queryPageIds()))

x=0
for p in pageids:
    x = x + 1
    print(x)
    print(p)
    payload = {'pageids': int(p), 'format': 'json', 'action': 'query', 'prop': 'extracts'}
    res = req(payload)
    updateLocation(res)


#print(res)





{'or_s': 41.993219, 'or_n': 46.306281, 'or_w': -124.7595306, 'or_e': -116.409837}
1
17006593
2
37699593
3
28639242
4
32464907
5
22650892
6
4138329
7
8142873
8
44678231
9
20413103
10
49979420
11
34701341
12
5316640
13
2309105
14
10608677
15
1531942
16
32022568
17
24535081
18
45039658
19
9748523
20
25485356
21
22473394
22
130937
23
4009992
24
18653234
25
29220915
26
3633767
27
48480313
28
26247123
29
35905595
30
32022590
31
32025269
32
15441986
33
22364229
34
47587398
35
6283335
36
44023881
37
32025271
38
32022604
39
26460237
40
30031950
41
48775247
42
38051923
43
1826900
44
45318229
45
3235928
46
32022619
47
52666460
48
1802333
49
39510111
50
39406267
51
13287524
52
32464998
53
9424913
54
22095459
55
32022634
56
49045614
57
15573105
58
44310643
59
41590900
60
32022647
61
130902
62
35295252
63
13320314
64
36577403
65
1654909
66
52060286
67
32120959
68
778369
69
22855811
70
15990916
71
32022661
72
18653318
73
27615371
74
15990925
75
22323346
76
32022676
77
28648716
78
49295385
79
10174616

652
28648758
653
7944857
654
37233976
655
14374452
656
23905596
657
9094465
658
32025483
659
41575748
660
689479
661
22193085
662
49513802
663
32030441
664
4138316
665
46523728
666
52069713
667
5752147
668
15033688
669
45056228
670
7963994
671
16172379
672
27601765
673
33949024
674
28648803
675
5981541
676
968047
677
130709
678
6587763
679
2114932
680
28648823
681
13575548
682
4728192
683
30860673
684
12779635
685
33916292
686
31352197
687
37979529
688
689551
689
24249744
690
26215827
691
23905688
692
42239385
693
130733
694
4457884
695
9407386
696
1749402
697
22193570
698
33752483
699
7169445
700
2114982
701
34309546
702
5850540
703
49391021
704
15175923
705
41559477
706
288182
707
52864755
708
41772472
709
25732537
710
46405197
711
6948286
712
24315328
713
6366659
714
37971396
715
2516422
716
44325793
717
7034444
718
6202826
719
5747577
720
130985
721
26355155
722
26363349
723
7705883
724
53111716
725
8226383
726
533987
727
16229862
728
42706411
729
27682285
730
24249838
731
4687348


1284
764477
1285
2157118
1286
20146752
1287
489227
1288
7105095
1289
10988104
1290
10627659
1291
18975309
1292
53373525
1293
29542998
1294
5540439
1295
32074329
1296
19663450
1297
32082523
1298
43786682
1299
10996319
1300
24423012
1301
29346405
1302
13068904
1303
32074347
1304
6728300
1305
7236205
1306
41093742
1307
739952
1308
26783486
1309
26086007
1310
4869257
1311
16190073
1312
32025211
1313
739964
1314
5753469
1315
6269567
1316
32025217
1317
8759938
1318
32074371
1319
35294316
1320
10472075
1321
32082573
1322
44575375
1323
32025232
1324
6343314
1325
21375635
1326
12954260
1327
32074389
1328
18639511
1329
32025242
1330
26811503
1331
1288863
1332
41618080
1333
32025249
1334
32475810
1335
32042489
1336
2534054
1337
32025255
1338
32025258
1339
32074414
1340
38931119
1341
33532594
1342
6171763
1343
5753524
1344
25250485
1345
445110
1346
20843191
1347
21899960
1348
32025275
1349
32074431
1350
32025284
1351
8817349
1352
32025289
1353
6695626
1354
32074443
1355
32040738
1356
5933775
1357


1887
27602686
1888
708351
1889
42569474
1890
35114757
1891
708358
1892
708361
1893
19967754
1894
708364
1895
21720846
1896
130791
1897
18534163
1898
18624277
1899
13531778
1900
708376
1901
42012442
1902
15011612
1903
708382
1904
2517093
1905
23792835
1906
48590627
1907
18607908
1908
708389
1909
6393638
1910
10579751
1911
32608041
1912
19902251
1913
25440044
1914
10407725
1915
29445936
1916
37245036
1917
19230516
1918
30232374
1919
39661370
1920
30136287
1921
578186
1922
44928830
1923
32025456
1924
18894660
1925
48000839
1926
11292488
1927
47460169
1928
12117948
1929
18624332
1930
6270798
1931
708433
1932
15005667
1933
33959765
1934
18886486
1935
23629655
1936
10189376
1937
47460186
1938
18607965
1939
8224420
1940
708448
1941
33640290
1942
3517755
1943
16756581
1944
17092457
1945
17133418
1946
44996583
1947
150980
1948
7032686
1949
47460207
1950
18583408
1951
12554097
1952
52604786
1953
18624372
1954
31362933
1955
42528634
1956
708475
1957
1421973
1958
130961
1959
49432416
1960
46657410

2487
7511534
2488
8950679
2489
15487897
2490
6255516
2491
40784799
2492
5845920
2493
18469793
2494
130830
2495
1831843
2496
32042481
2497
23507880
2498
51270569
2499
47813547
2500
9409452
2501
10146733
2502
7867723
2503
17305953
2504
152498
2505
18637215
2506
6135865
2507
37278646
2508
51073979
2509
19051338
2510
35405301
2511
43176896
2512
47150019
2513
9688004
2514
23793142
2515
12555209
2516
20362187
2517
25961293
2518
30479315
2519
17216470
2520
25738745
2521
4688755
2522
766943
2523
49411040
2524
23974884
2525
4137392
2526
6812648
2527
11640316
2528
26391530
2529
6812651
2530
49763311
2531
709616
2532
6891688
2533
18469874
2534
23507957
2535
47799807
2536
24163328
2537
2102273
2538
766979
2539
50885636
2540
28627976
2541
50885641
2542
3945483
2543
16364556
2544
50885645
2545
5846030
2546
20760749
2547
52851544
2548
32779779
2549
36443157
2550
52974615
2551
8778776
2552
18879513
2553
30290972
2554
11424797
2555
32453662
2556
34149412
2557
23540773
2558
2700330
2559
32453676
2560
10

3091
6568127
3092
2005184
3093
1756875
3094
6224068
3095
2005190
3096
19227681
3097
130764
3098
36526283
3099
42825934
3100
22960337
3101
13097170
3102
1243348
3103
130766
3104
39859236
3105
130767
3106
13007068
3107
32041338
3108
130768
3109
36526307
3110
415978
3111
7685159
3112
34535660
3113
23517421
3114
15472878
3115
47749359
3116
23927024
3117
8817298
3118
20119251
3119
14675912
3120
36526325
3121
1693942
3122
6997033
3123
29220949
3124
24172800
3125
2054404
3126
23345415
3127
36526344
3128
46588194
3129
24181004
3130
702734
3131
18618640
3132
24075992
3133
20560259
3134
32586007
3135
33356056
3136
38041882
3137
743707
3138
130778
3139
24516894
3140
41154949
3141
36526368
3142
33921313
3143
10227419
3144
22772010
3145
32041351
3146
24246572
3147
33921326
3148
28817711
3149
30914952
3150
2226483
3151
19905155
3152
28129589
3153
6983720
3154
20363576
3155
20748601
3156
481600
3157
32041355
3158
5454150
3159
745185
3160
25540936
3161
52812105
3162
20142412
3163
21903015
3164
1043489

3694
36961616
3695
28638545
3696
19905876
3697
19905877
3698
19905881
3699
24172873
3700
13006053
3701
9764192
3702
32030608
3703
40449254
3704
19905895
3705
33512808
3706
19905897
3707
23682411
3708
19905900
3709
19905902
3710
32054639
3711
15957360
3712
26811761
3713
5725555
3714
19905908
3715
31186294
3716
25812344
3717
130964
3718
19905915
3719
28648682
3720
19905918
3721
52166015
3722
23518592
3723
48897409
3724
28630402
3725
19905924
3726
19905927
3727
18701704
3728
23485833
3729
19905931
3730
15482255
3731
376209
3732
19905938
3733
18218387
3734
44539285
3735
19905942
3736
23879577
3737
19905944
3738
19905946
3739
17694107
3740
33422749
3741
19905954
3742
130971
3743
11484580
3744
4881830
3745
10730919
3746
19905960
3747
130972
3748
44080554
3749
2276780
3750
46496033
3751
23485871
3752
19905968
3753
130974
3754
35289673
3755
19905976
3756
19905979
3757
19905981
3758
32447934
3759
35724735
3760
21298625
3761
11769885
3762
14065092
3763
26811846
3764
25821089
3765
3276232
3766
44